In [3]:
import numpy as np 
import cv2
from tqdm import tqdm
#
import tensorflow as tf 
from tensorflow.keras import layers, models
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.preprocessing.image import  ImageDataGenerator
from sklearn.model_selection import train_test_split
#
import os
import sys
#
import gtls
from episodic_gwr import EpisodicGWR

In [4]:
def load_pre_trained(input_data):
    vgg_net = VGG16(include_top=False,weights='imagenet')
    vgg_net.trainable = False
    x = vgg_net(input_data)
    #x = layers.Conv2D(256,(1,1),activation='relu')(x)
    #x = layers.MaxPool2D((2,2))(x)
    #x = layers.Conv2D(256,(1,1),activation='relu')(x)
    x = layers.MaxPool2D((4,4))(x)
    x = layers.Flatten()(x)
    #x = layers.Dense(256,activation='relu')(x)
    return x

def get_data():
    dataset_path = "../../DMRSON_dataset_files/dataset/core50_128x128/core50_128x128/"
    working_dir = os.path.abspath(os.path.join('..'))
    non_training_scenes = ['s3','s7','s10']
    training_scenes = ['s1']
    if working_dir not in sys.path:
        sys.path.append(working_dir)
    data = []
    objects = []
    labels = []
    images = []
    for s in tqdm(os.listdir(dataset_path)):
        if s in training_scenes:
            for d in os.listdir(dataset_path+'/'+s):
                objects.append(d)
                for img in os.listdir(dataset_path+'/'+s+'/'+d):
                    images = cv2.resize(cv2.imread(dataset_path+'/'+s+'/'+d+'/'+img),(244,244)).astype(np.float32)
                    labels = objects.index(d)
                    data.append([images,labels])
            objects = []
    #print(np.shape(data))
    #print(data)
    
    #return np.array(images,dtype=object), np.array(labels,dtype=object)
    return np.array(data,dtype=object)

def write_file(file,value):
    # save the numpy array 
    if not file in os.listdir():
        with open(file,'wb') as f:
            np.save(f,value)
            print("Features stored ")

def load_file(file):
    # load features 
    with open(file,'rb') as f:
        value = np.load(f)
        print("Features readed")
    
    return value



In [5]:
extract_features = True

dataset = get_data()
#data = np.array([d[0] for d in dataset])
#labels = np.array([l[1]for l in dataset])
print(np.shape(dataset))
#print(np.shape(data))
#print(np.shape(labels))

100%|██████████| 11/11 [00:13<00:00,  1.20s/it]


In [1]:
e_label = [50,50]
s_label = [10]


num_of_classes = e_label[0]

#(x_train,x_val,y_train,y_val) = train_test_split(data,labels,test_size=0.0,random_state=None, shuffle=False)
x_train = data/255.0
y_train = labels
x_val = 0
y_val = 0

print(f"x train shape : {np.shape(x_train)}")
print(f"y train shape : {np.shape(y_train)}")
print(f"x validation shape : {np.shape(x_val)}")
print(f"y validation shape : {np.shape(y_val)}")


# normalize the data 
#x_train = x_train/255
#x_val = x_val/255

x_train = np.asarray(x_train).astype('float32')
x_val = np.asarray(x_val).astype('float32')
y_train = np.asarray(y_train).astype('float32')
y_val = np.asarray(y_val).astype('float32')

#print(y_train)
#y_train = tf.keras.utils.to_categorical(y_train, num_of_classes)
#y_val = tf.keras.utils.to_categorical(y_val, num_of_classes)
#print(y_train)

NameError: name 'data' is not defined

In [1]:
print(y_train)
x_train.shape[3]
#VGG16().summary()

NameError: name 'y_train' is not defined

In [2]:
# model construction
input_data = tf.keras.Input(shape=(128,128,3))
# pre-trained cnn
x = load_pre_trained(input_data)

model = tf.keras.Model(inputs=input_data,outputs=x)
model.summary()

NameError: name 'tf' is not defined

In [3]:
# feature extractor
# uncommnet to generate the feature extraction 
if extract_features == True:
    x_train_features = np.zeros((x_train.shape[0],512))
    for i in range(x_train.shape[0]):
        image = x_train[i]
        image = image.reshape((1,image.shape[0],image.shape[1],image.shape[2]))
        x_train_features[i] = model.predict(image)
    x_train_features.shape


NameError: name 'extract_features' is not defined

In [10]:
# GDM params 
num_context = 1
d_labels = np.zeros((len(e_label),len(y_train)))
d_labels[0] = y_train
d_labels[1] = y_train
epochs = 5
a_threshold = [0.3,0.001]
beta = 0.7
learning_rate = [0.5,0.005]
context = True

In [11]:

# episodic memory -- init
g_episodic = EpisodicGWR()
g_episodic.init_network(x_train_features,e_label,num_context=num_context)

# semantic memory -- init
g_semantic = EpisodicGWR()
g_semantic.init_network(x_train_features,s_label,num_context=num_context)

# batch learning 
e_NN,e_qerr = g_episodic.train_egwr(x_train_features,d_labels,epochs, a_threshold[0], beta, learning_rate, context, regulated=0)

11991


IndexError: index 24 is out of bounds for axis 1 with size 10

In [ ]:
"""
print(x_train[0].shape)

#baseModel = VGG16()
baseModel = VGG16(weights="imagenet", include_top=False,
	input_tensor=tf.keras.Input(shape=(128, 128, 3)))
#baseModel = layers.Flatten()(baseModel)
#baseModel.summary()

model = tf.keras.Model(inputs=baseModel.inputs, outputs=baseModel.layers[-2].output)

image = x_train[0]
image = image.reshape((1,image.shape[0],image.shape[1],image.shape[2]))

res = model.predict(image)
res.shape
"""